# APIM ❤️ OpenAI

## Tracing tool

Invoke OpenAI API with trace enabled and returns the Azure API Management [tracing information](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-api-inspector). Tracing helps you debug and troubleshoot your API.

## TOC
- [Initialize notebook variables](#0)
- [Get the deployment outputs](#1)
- [Get the APIM authorization debug token](#2)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🔍 Analyze the API trace from direct HTTP call](#trace1)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🔍 Analyze the API trace from the SDK call](#trace2)


<a id='0'></a>
### Initialize notebook variables

In [ ]:
deployment_name = "" # name of the label that you want to use with  this tool (ex: semantic-caching)
resource_group_name = f"lab-{deployment_name}"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"

<a id='1'></a>
### Get the deployment outputs

In [ ]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = deployment_stdout.n
print("👉🏻 APIM Service Id: ", apim_service_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)


<a id='2'></a>
### Get the APIM authorization debug token

This token will be used to trace the API request.

In [ ]:
import requests
import json
token = ! az account get-access-token --query accessToken --output tsv

request = {
    "credentialsExpireAfter": "PT1H",
    "apiId": apim_service_id + "/apis/openai",
    "purposes": ["tracing"]
}
url = "https://management.azure.com" + apim_service_id + "/gateways/managed/listDebugCredentials?api-version=2023-05-01-preview"

response = requests.post(url, headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token.n}, json = request)

if (response.status_code == 200):
    data = json.loads(response.text)
    apim_debug_authorization = data.get("token")
else:
    print(response.text)


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [ ]:
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = {'api-key':apim_subscription_key, 'Apim-Debug-Authorization': apim_debug_authorization}, json = messages)
print("status code: ", response.status_code)
trace_id = response.headers.get("Apim-Trace-Id")
print("Apim-Trace-Id: ", trace_id) # this header will be used to get API trace details
print("headers ", response.headers)
print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
print("x-ms-stream: ", response.headers.get("x-ms-stream")) # this header is useful to determine if the response is streamed
if (response.status_code == 200):
    data = json.loads(response.text)
    print("💬 ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)


<a id='trace1'></a>
### 🔍 Analyze the API trace from direct HTTP call

With the following request we will get the json with the complete trace information.

In [ ]:
request = {
    "traceId": trace_id
}
url = "https://management.azure.com" + apim_service_id + "/gateways/managed/listTrace?api-version=2023-05-01-preview"
response = requests.post(url, headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token.n}, json = request)

if (response.status_code == 200):
    data = json.loads(response.text)
    print(json.dumps(data, indent=4))
else:
    print(response.text)


<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK


In [ ]:
import time
from openai import AzureOpenAI
messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)
response = client.chat.completions.with_raw_response.create(model=openai_deployment_name, messages=messages, extra_headers={'Apim-Debug-Authorization': apim_debug_authorization})
trace_id = response.headers.get("Apim-Trace-Id")
print("Apim-Trace-Id: ", trace_id) # this header will be used to get API trace details
print("headers ", response.headers)
print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
print("x-ms-stream: ", response.headers.get("x-ms-stream")) # this header is useful to determine if the response is streamed

completion = response.parse() 
print("💬 ",completion.choices[0].message.content)


<a id='trace2'></a>
### 🔍 Analyze the API trace from the SDK call

In [ ]:
request = {
    "traceId": trace_id
}
url = "https://management.azure.com" + apim_service_id + "/gateways/managed/listTrace?api-version=2023-05-01-preview"
response = requests.post(url, headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token.n}, json = request)

if (response.status_code == 200):
    data = json.loads(response.text)
    print(json.dumps(data, indent=4))
else:
    print(response.text)
